# 출력 파서 (`OutputParser`)

- LLM의 출력을 더 유용/구조화된 형태로 변환함
- 구조화: LLM의 자유 형식 출력을 구조화된 데이터로 변환
- 일관성: 항상 일관된 출력 형식 -> 후속 처리에 용이
- 유연성: 다양한 출력 형식(JSON, list, dict)으로 변환 가능

# 출력 파서 (`OutputParser`)

- LLM의 출력을 더 유용/구조화된 형태로 변환함
- 구조화: LLM의 자유 형식 출력을 구조화된 데이터로 변환
- 일관성: 항상 일관된 출력형식 -> 후속 처리에 용이
- 유연성: 다양한 출력 형식(JSON, list, dict)으로 변환 가능

## `PydanticOutputParser`

In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(model='gpt-4.1-nano')

In [4]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [5]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    '아래 이메일 내용중 중요한 것만 추출해.\n\n---\n\n{email_conversation}'
)

# print(prompt.format(email_conversation=email_conversation))
chain = prompt | llm 

print('---출력파서 없는 요약---')
print(chain.invoke({'email_conversation': email_conversation}).content)

---출력파서 없는 요약---
아래는 이메일에서 중요한 내용입니다:

1. 발신자: 김철수 상무 (바이크코퍼레이션)
2. 수신자: 이은채 대리
3. 목적:
   - "ZENESIS" 자전거에 대한 상세 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 필요)
   - 유통 전략 및 마케팅 계획 수립을 위한 자료 요청
4. 미팅 제안:
   - 일정: 1월 15일 화요일 오전 10시
   - 장소: 귀사 사무실에서 미팅 희망


In [6]:
class EmailSummary(BaseModel):
    person: str = Field(description='메일 보낸 사람')
    email: str = Field(description='보낸사람의 메일주소')
    subject: str = Field(description='메일 제목')
    summary: str = Field(description='메일 본문 요약')
    date: str = Field(description='메일에 언급된 미팅 날짜와 시간')

parser = PydanticOutputParser(pydantic_object=EmailSummary)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "보낸사람의 메일주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문 요약", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [7]:
prompt = PromptTemplate.from_template(
    """
너는 요약의 신 어시스턴트야. 아래 질문에 맞게 답변을 한국어로 만들어줘

질문: {question}

이메일 내용: {email_conversation}

형식: {format}

"""
)

# 프롬프트 변수들중 일부만 채우기
prompt = prompt.partial(format=parser.get_format_instructions())

In [8]:
chain = prompt | llm | parser

res = chain.invoke(
    {
        'question': '이메일 내용중 중요한 내용을 추출해 줘!',
        'email_conversation': email_conversation
    }
)

res

EmailSummary(person='김철수 상무', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션은 귀사의 "ZENESIS" 자전거에 대한 브로슈어 요청과 함께, 기술 사양, 배터리 성능, 디자인 정보를 요구하며, 다음 주 1월 15일 오전 10시에 사무실에서 협력 관련 미팅을 제안하고 있습니다.', date='2023년 1월 15일 오전 10시')

`PydanticOutputParser`

In [9]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import Base

ImportError: cannot import name 'Base' from 'pydantic' (c:\Users\mingyu\TIL\LLM-agent\.venv\Lib\site-packages\pydantic\__init__.py)

In [ ]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    '아래 이메일 내용중 중요한 것만 추출해.{email_conversation}'
)

# print(prompt.format(email_conversation = email_conversation))
chain = prompt | llm 

print('---출력파서 없는 요약---')
print(chain.invoke({'email_conversation': email_conversation}).content)

In [ ]:
class EmailSummary(BaseModel):
    person: str
    email: str
    subject: str
    summary: str
    date: str

parser = PydanticOutputParser(pydantic_object = EmailSummary)

In [ ]:
chain = prompt | llm |parser
print(chain.invoke({'email_conversation':email_}))

In [ ]:
prompt = PromptTemplate.from_template(
    """
너는 요약의 신 어시스턴트야. 아래 질문에 맞게 

질문: {question}


"""
)

# 프롬프트 변수들 중 일부만 채우기
prompt.partial(format=parser.get_format_instructions())

# csv parser

In [10]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

output_parser.get_format_instructions()

prompt = PromptTemplate(
    template = 'List 5 {subject}. \n{format_instructions}',
    input_variables = ['subject'],
    partial_variables = {'format_instructions':output_parser.get_format_instructions()}    
)
print('---')
print(prompt.format(subject = '중국집대표메뉴'))

chain1 = prompt | llm
csv_res = chain1.invoke({'subject':'맥도날드 대표메뉴'}).content

chain2 = prompt | llm | output_parser
list_res = chain1.invoke({'subject': '국민주식'})

---
List 5 중국집대표메뉴. 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [11]:
print(csv_res)
print(list_res)

Big Mac, Quarter Pounder, McNuggets, Filet-O-Fish, Fries
content='삼성전자, SK하이닉스, 현대차, LG화학, 네이버' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 43, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_c4c155951e', 'id': 'chatcmpl-CBCnzOUhOATcuQcYs3ASI2Y3yHbAj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--63c0433c-2652-4a54-ada2-9d5d200bf1a4-0' usage_metadata={'input_tokens': 43, 'output_tokens': 19, 'total_tokens': 62, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


# Structured Output Parser

- dict 형식 -> 멍청한 모델에도 적용 가능

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser 

In [ ]:
response_schema = [
    ResponseSchema(name = 'answer',type = 'str', description='사용자의 질문에 대한 답변'),
    ResponseSchema(name = 'source',description='질문에 답하기 위해 사용된 "출처","웹사이트 주소"')
]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)


In [ ]:
print(output_parser.get_format_instructions())

prompt = PromptTemplate(
    template = '사용자 질문에 최선을 다해 답변한다.\n{format_instructions}\n{question}',
    input_variables=['question'],
    partial_variables={'format_instructions'}
)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [14]:
%pip install -q seaborn

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain.output_parsers import PandasDataFrameOutputParser

import seaborn as sns

titanic_df = sns.load_dataset('titanic')

ModuleNotFoundError: No module named 'seaborn'

In [17]:
prompt = PromptTemplate(
    template = 'Answer the user query.\n{format_instructions}\n{query}',
    input_variables=['query'],
    partial_variables={'format_instructions': parser.get_format_instructions()},
)
res = prompt.format(query= 'Age 컬럼의 평균은 얼마냐?')
print(res)

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "보낸사람의 메일주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문 요약", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```
Age 컬럼의 평균은 얼마냐?


In [ ]:
prompt = PromptTemplate(
    template = 'Answer the user query.\n{}'
)

In [21]:
from langchain_openai import ChatOpenAI
msg = input('외국어를 넣어봐')
llm = ChatOpenAI(model = 'gpt-4.1-nano')
res = llm.invoke(f'한국어로 번역해봐:{msg}')
res.content

'하이 하이'

In [24]:
## datetime Output Parser
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser(format = '%Y-%m-%d')
prompt = PromptTemplate(
    template = 'aaa',
    input_variables = ['question'],
    partial_variables= {
        'format_instruction':output_parser.get_format_instructions()
    }
)
res = output_parser.get_format_instructions()
res.contet

AttributeError: 'str' object has no attribute 'contet'

In [ ]:
from langchain
from enum import Enum

class Colors(Enum):
    RED = '빨간색'
    GREEN = '초록색'
    BLUE = '파란색'